In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!free -g

              total        used        free      shared  buff/cache   available
Mem:             12           1           0           0          10          11
Swap:             0           0           0


In [4]:
!pip3 install imageio

    100% |████████████████████████████████| 3.3MB 6.4MB/s 


In [0]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model

import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import imageio

In [0]:
img_rows = 56
img_cols = 56
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [0]:
def build_discriminator():
  
  model = Sequential()

  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(56,56,3), padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)

In [0]:
def build_generator():

  model = Sequential()

  model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
  model.add(Reshape((7, 7, 128)))
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  #         

  model.add(UpSampling2D())
  model.add(Conv2D(32, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  #         
  model.add(Conv2D(channels, kernel_size=3, padding="same"))
  model.add(Activation("tanh"))

  #         model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(noise, img)


In [0]:
def model_train(epochs,discriminator,generator, combined_model, batch_size=64, save_interval=1):

  # Load the dataset
  path = 'drive/projects/SpriteGAN/pokemon_sprites'

  nb_samples = len(os.listdir(path))
  #         print(nb_samples)

  X_train = np.zeros(shape=(nb_samples, img_rows, img_cols,3), dtype= 'float32')

  for i,f in enumerate(os.listdir(path)):
    img_path = os.path.join(path, f)
    img = imageio.imread(img_path)
    img_no_alpha = img[:,:,:3]
    if img_no_alpha.shape != X_train[0].shape:
      npad = ((int((img_rows-img_no_alpha.shape[0])/2), int((img_rows-img_no_alpha.shape[0])/2)), \
              (int((img_cols-img_no_alpha.shape[1])/2), int((img_cols-img_no_alpha.shape[1])/2)), (0, 0))
      padded_img = np.pad(img_no_alpha, pad_width=npad, mode='constant', constant_values=255)
      X_train[i] = padded_img
      continue
    X_train[i] = img_no_alpha          



  # Rescale -1 to 1
  X_train = X_train / 255.
  #         X_train = np.expand_dims(X_train, axis=3)

  # Adversarial ground truths
  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):

      # ---------------------
      #  Train Discriminator
      # ---------------------
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      # Sample noise and generate a batch of new images
      noise = np.random.normal(-1, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      # Train the discriminator (real classified as ones and generated as zeros)
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      # ---------------------
      #  Train Generator
      # ---------------------

      # Train the generator (wants discriminator to mistake images as real)
      g_loss = combined_model.train_on_batch(noise, valid)

      # Plot the progress
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

      # If at save interval => save generated image samples
      if epoch % save_interval == 0:
          save_imgs(epoch,generator)

In [0]:
def save_imgs(epoch,generator):
  n_imgs  = 4
  
  fig = plt.figure()

  for i in range(n_imgs):
    noise = np.random.normal(-1, 1, (1, latent_dim))
    gen_imgs = generator.predict(noise)
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.imshow(np.clip(gen_imgs[0],0,1))

#   plt.show()
  plt.savefig("drive/projects/SpriteGAN/Images/spriteGANFinal_%d.png" % epoch)
  plt.close()

In [33]:
optimizer = Adam(0.0002, 0.5)
# Build and compile the discriminator
path = "drive/projects/SpriteGAN/"
use_prev = False
if os.path.exists(os.path.join(path,'discriminator.h5')) and use_prev:
  print("Loading previous discriminator..")
  discriminator = load_model(os.path.join(path,'discriminator.h5'))
else:
  print("Created new discriminator..")
  discriminator = build_discriminator()
  
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
if os.path.exists(os.path.join(path,'generator.h5')) and use_prev:
  print("Loading previous generator..")
  generator = load_model(os.path.join(path,'generator.h5'))
else:
  print("Created new generator..")
  generator = build_generator()


# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
if os.path.exists(os.path.join(path,'combined_model.h5')) and use_prev:
  print("Loading previous combined model..")
  combined_model = load_model(os.path.join(path,'combined_model.h5'))
else:
  print("Created new Combined Model..")
  combined_model = Model(z, valid)
  
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer)

Created new discriminator..
Created new generator..
Created new Combined Model..


In [0]:
model_train(7000,discriminator,generator, combined_model,save_interval=500)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.160512, acc.: 35.94%] [G loss: 0.812309]
1 [D loss: 0.778889, acc.: 52.34%] [G loss: 1.271092]
2 [D loss: 0.552109, acc.: 69.53%] [G loss: 1.782951]
3 [D loss: 0.574576, acc.: 67.19%] [G loss: 2.368200]
4 [D loss: 0.423280, acc.: 84.38%] [G loss: 2.787068]
5 [D loss: 0.335677, acc.: 88.28%] [G loss: 2.674973]
6 [D loss: 0.222269, acc.: 95.31%] [G loss: 2.559963]
7 [D loss: 0.192443, acc.: 96.88%] [G loss: 2.474340]
8 [D loss: 0.193606, acc.: 96.88%] [G loss: 2.268680]
9 [D loss: 0.128030, acc.: 97.66%] [G loss: 1.884765]
10 [D loss: 0.121112, acc.: 97.66%] [G loss: 2.108341]
11 [D loss: 0.094490, acc.: 100.00%] [G loss: 2.063184]
12 [D loss: 0.144136, acc.: 98.44%] [G loss: 2.553777]
13 [D loss: 0.167909, acc.: 97.66%] [G loss: 3.767796]
14 [D loss: 0.173361, acc.: 100.00%] [G loss: 4.055923]
15 [D loss: 0.228554, acc.: 95.31%] [G loss: 5.171544]
16 [D loss: 0.225414, acc.: 95.31%] [G loss: 5.534538]
17 [D loss: 0.964700, acc.: 46.09%] [G loss: 7.667124]
18 [D loss: 0.8268

83 [D loss: 1.219174, acc.: 32.81%] [G loss: 1.617505]
84 [D loss: 1.097915, acc.: 33.59%] [G loss: 1.767023]
85 [D loss: 1.089468, acc.: 37.50%] [G loss: 1.544335]
86 [D loss: 1.081775, acc.: 29.69%] [G loss: 1.569162]
87 [D loss: 1.144340, acc.: 28.91%] [G loss: 1.439778]
88 [D loss: 1.162905, acc.: 35.16%] [G loss: 1.623634]
89 [D loss: 1.287718, acc.: 19.53%] [G loss: 1.942110]
90 [D loss: 1.107558, acc.: 29.69%] [G loss: 1.661081]
91 [D loss: 1.241186, acc.: 28.12%] [G loss: 1.646648]
92 [D loss: 1.090767, acc.: 38.28%] [G loss: 1.821788]
93 [D loss: 0.864384, acc.: 45.31%] [G loss: 1.759146]
94 [D loss: 1.109121, acc.: 26.56%] [G loss: 1.469223]
95 [D loss: 1.033199, acc.: 37.50%] [G loss: 1.545547]
96 [D loss: 1.099605, acc.: 33.59%] [G loss: 1.803636]
97 [D loss: 1.294542, acc.: 19.53%] [G loss: 1.665433]
98 [D loss: 1.265456, acc.: 20.31%] [G loss: 1.833154]
99 [D loss: 1.239634, acc.: 28.91%] [G loss: 1.716477]
100 [D loss: 1.237986, acc.: 23.44%] [G loss: 1.771568]
101 [D lo

165 [D loss: 1.138434, acc.: 35.16%] [G loss: 1.719876]
166 [D loss: 1.109964, acc.: 38.28%] [G loss: 1.665375]
167 [D loss: 1.193846, acc.: 27.34%] [G loss: 1.614560]
168 [D loss: 1.173823, acc.: 28.12%] [G loss: 1.348408]
169 [D loss: 1.129404, acc.: 31.25%] [G loss: 1.437469]
170 [D loss: 1.054742, acc.: 35.16%] [G loss: 1.652817]
171 [D loss: 0.856508, acc.: 46.09%] [G loss: 1.772604]
172 [D loss: 0.800686, acc.: 56.25%] [G loss: 1.948814]
173 [D loss: 0.959884, acc.: 41.41%] [G loss: 1.939966]
174 [D loss: 0.720090, acc.: 64.06%] [G loss: 1.962964]
175 [D loss: 0.644521, acc.: 64.84%] [G loss: 2.031033]
176 [D loss: 0.543810, acc.: 74.22%] [G loss: 2.174466]
177 [D loss: 0.730281, acc.: 57.81%] [G loss: 2.355108]
178 [D loss: 0.608992, acc.: 68.75%] [G loss: 2.296621]
179 [D loss: 0.597983, acc.: 68.75%] [G loss: 2.412348]
180 [D loss: 1.004748, acc.: 42.19%] [G loss: 2.486513]
181 [D loss: 0.931166, acc.: 44.53%] [G loss: 2.246013]
182 [D loss: 1.132013, acc.: 35.16%] [G loss: 2.

247 [D loss: 0.696766, acc.: 62.50%] [G loss: 2.000910]
248 [D loss: 1.023490, acc.: 35.94%] [G loss: 2.311222]
249 [D loss: 0.841627, acc.: 48.44%] [G loss: 2.091538]
250 [D loss: 0.867807, acc.: 51.56%] [G loss: 2.187047]
251 [D loss: 0.934103, acc.: 41.41%] [G loss: 2.074834]
252 [D loss: 0.785058, acc.: 53.91%] [G loss: 1.765121]
253 [D loss: 0.924523, acc.: 48.44%] [G loss: 1.656764]
254 [D loss: 0.901073, acc.: 50.78%] [G loss: 1.773949]
255 [D loss: 0.906579, acc.: 49.22%] [G loss: 1.879868]
256 [D loss: 0.852414, acc.: 51.56%] [G loss: 2.120039]
257 [D loss: 0.852185, acc.: 47.66%] [G loss: 1.960437]
258 [D loss: 1.247507, acc.: 32.03%] [G loss: 1.848269]
259 [D loss: 1.153311, acc.: 32.81%] [G loss: 2.070713]
260 [D loss: 1.364501, acc.: 24.22%] [G loss: 2.187680]
261 [D loss: 1.287769, acc.: 20.31%] [G loss: 2.309474]
262 [D loss: 1.189383, acc.: 30.47%] [G loss: 2.199387]
263 [D loss: 1.177267, acc.: 38.28%] [G loss: 2.737117]
264 [D loss: 1.083133, acc.: 42.19%] [G loss: 2.

...

575 [D loss: 0.635440, acc.: 67.97%] [G loss: 3.008015]
576 [D loss: 0.760671, acc.: 57.03%] [G loss: 3.641909]
577 [D loss: 0.422806, acc.: 81.25%] [G loss: 3.177104]
578 [D loss: 0.590197, acc.: 71.09%] [G loss: 3.665233]
579 [D loss: 0.751284, acc.: 60.16%] [G loss: 3.306874]
580 [D loss: 0.710910, acc.: 57.03%] [G loss: 3.530002]
581 [D loss: 0.634483, acc.: 65.62%] [G loss: 3.029847]
582 [D loss: 0.794084, acc.: 59.38%] [G loss: 3.326308]
583 [D loss: 0.747281, acc.: 59.38%] [G loss: 3.299923]
584 [D loss: 0.683626, acc.: 62.50%] [G loss: 3.494052]
585 [D loss: 0.705279, acc.: 65.62%] [G loss: 3.976614]
586 [D loss: 0.573838, acc.: 71.09%] [G loss: 3.271725]
587 [D loss: 0.748478, acc.: 64.84%] [G loss: 3.724338]
588 [D loss: 0.686679, acc.: 65.62%] [G loss: 3.177717]
589 [D loss: 0.820663, acc.: 54.69%] [G loss: 3.566956]
590 [D loss: 0.791937, acc.: 63.28%] [G loss: 3.175691]
591 [D loss: 0.751103, acc.: 56.25%] [G loss: 3.203807]
592 [D loss: 0.580804, acc.: 65.62%] [G loss: 3.

657 [D loss: 0.695016, acc.: 68.75%] [G loss: 3.002402]
658 [D loss: 0.406115, acc.: 85.16%] [G loss: 3.178471]
659 [D loss: 0.574515, acc.: 70.31%] [G loss: 3.352828]
660 [D loss: 0.885617, acc.: 48.44%] [G loss: 3.087251]
661 [D loss: 0.613455, acc.: 65.62%] [G loss: 3.741837]
662 [D loss: 0.735083, acc.: 58.59%] [G loss: 3.388144]
663 [D loss: 0.816545, acc.: 53.91%] [G loss: 3.949585]
664 [D loss: 0.423561, acc.: 79.69%] [G loss: 3.486484]
665 [D loss: 0.799810, acc.: 54.69%] [G loss: 4.655039]
666 [D loss: 0.585873, acc.: 70.31%] [G loss: 3.642529]
667 [D loss: 0.518050, acc.: 73.44%] [G loss: 3.806787]
668 [D loss: 0.524357, acc.: 74.22%] [G loss: 4.198163]
669 [D loss: 0.638203, acc.: 67.97%] [G loss: 3.535824]
670 [D loss: 0.681823, acc.: 59.38%] [G loss: 4.048678]
671 [D loss: 0.779546, acc.: 59.38%] [G loss: 4.192977]
672 [D loss: 0.669653, acc.: 60.16%] [G loss: 3.797333]
673 [D loss: 0.646880, acc.: 65.62%] [G loss: 3.495361]
674 [D loss: 0.665478, acc.: 64.84%] [G loss: 3.

739 [D loss: 0.498702, acc.: 80.47%] [G loss: 3.643458]
740 [D loss: 0.813918, acc.: 53.12%] [G loss: 3.481195]
741 [D loss: 0.506063, acc.: 78.12%] [G loss: 3.252787]
742 [D loss: 0.556835, acc.: 68.75%] [G loss: 3.547526]
743 [D loss: 0.706789, acc.: 63.28%] [G loss: 3.494626]
744 [D loss: 0.716676, acc.: 61.72%] [G loss: 3.302740]
745 [D loss: 0.525327, acc.: 72.66%] [G loss: 3.269603]
746 [D loss: 0.478789, acc.: 78.12%] [G loss: 3.164661]
747 [D loss: 0.512932, acc.: 75.00%] [G loss: 2.974379]
748 [D loss: 0.629680, acc.: 63.28%] [G loss: 3.267895]
749 [D loss: 0.629448, acc.: 65.62%] [G loss: 2.879060]
750 [D loss: 0.600717, acc.: 67.97%] [G loss: 3.209656]
751 [D loss: 0.609483, acc.: 67.97%] [G loss: 2.848450]
752 [D loss: 0.822946, acc.: 50.78%] [G loss: 3.488279]
753 [D loss: 0.609346, acc.: 74.22%] [G loss: 3.461962]
754 [D loss: 0.713375, acc.: 59.38%] [G loss: 2.641818]
755 [D loss: 0.540916, acc.: 71.09%] [G loss: 3.308851]
756 [D loss: 0.451174, acc.: 79.69%] [G loss: 3.

821 [D loss: 0.850359, acc.: 59.38%] [G loss: 4.333930]
822 [D loss: 0.679849, acc.: 66.41%] [G loss: 3.990471]
823 [D loss: 1.156401, acc.: 35.16%] [G loss: 3.785904]
824 [D loss: 0.897844, acc.: 52.34%] [G loss: 3.143878]
825 [D loss: 0.710900, acc.: 67.19%] [G loss: 2.716749]
826 [D loss: 0.598812, acc.: 66.41%] [G loss: 3.639431]
827 [D loss: 0.413143, acc.: 81.25%] [G loss: 3.440727]
828 [D loss: 0.418881, acc.: 84.38%] [G loss: 2.829215]
829 [D loss: 0.467102, acc.: 76.56%] [G loss: 3.350882]
830 [D loss: 0.321100, acc.: 90.62%] [G loss: 3.386025]
831 [D loss: 0.592314, acc.: 68.75%] [G loss: 3.281096]
832 [D loss: 0.448774, acc.: 80.47%] [G loss: 3.497530]
833 [D loss: 0.583102, acc.: 67.19%] [G loss: 3.595384]
834 [D loss: 0.550413, acc.: 71.88%] [G loss: 3.233295]
835 [D loss: 0.331796, acc.: 84.38%] [G loss: 3.491857]
836 [D loss: 0.460519, acc.: 77.34%] [G loss: 3.316384]
837 [D loss: 0.676461, acc.: 63.28%] [G loss: 3.419847]
838 [D loss: 0.485386, acc.: 75.00%] [G loss: 3.

903 [D loss: 0.753735, acc.: 61.72%] [G loss: 4.093732]
904 [D loss: 0.440327, acc.: 82.03%] [G loss: 3.415831]
905 [D loss: 0.514979, acc.: 75.78%] [G loss: 3.495476]
906 [D loss: 0.595545, acc.: 70.31%] [G loss: 3.458439]
907 [D loss: 0.710127, acc.: 62.50%] [G loss: 3.058052]
908 [D loss: 0.537221, acc.: 74.22%] [G loss: 3.793040]
909 [D loss: 0.316386, acc.: 88.28%] [G loss: 3.614679]
910 [D loss: 0.400768, acc.: 79.69%] [G loss: 3.191824]


In [0]:
discriminator.save(os.path.join(path,"discriminator4x4.h5"))

In [0]:
generator.save(os.path.join(path,"generator4x4.h5"))

In [0]:
combined_model.save(os.path.join(path,"combined_model4x4.h5"))